## Base de données JSON : MongoDB avancé

### Lecture de données

In [1]:
import csv

lignes = []
def get_ligne(id):
    for l in lignes:
        if l['_id'] == id:
            return l
    return None
    
# Lecture des informations
with open("ponctualite-mensuelle-transilien.csv",'r',encoding='ISO-8859-1') as csvfile:
    reader = csv.reader(csvfile, delimiter=';', quotechar='"')
    first_line = True
    for row in reader:
        if not first_line:
            (id,date,service,sigle,nom,ponctualite,satisfaction) = row
            ponctualite = float(ponctualite) if not ponctualite == '' else 0.0
            satisfaction = float(satisfaction)
            ligne = get_ligne(id)
            if not ligne:
                ligne = { "_id":id, "service":service, "sigle":sigle, "nom":nom, "mesures":[] }
                lignes.append(ligne)
            ligne['mesures'].append({'date': date, 'ponctualite': ponctualite, 'satisfaction': satisfaction})
        first_line = False

### Collections

In [3]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['test']

# Structure arborescente (liste des lignes, contenant les mesures)
transiliens = db['transiliens']

# Structure à plat (liste des mesures, avec indication des lignes)
mesures = db['transiliens:mesures']

<div id="alimentation_des_collections"></div>

### Alimentation des collections

In [4]:
for l in lignes:
    # structure arborescente
    transiliens.update_one( {'_id': l['_id']}, {'$set': l}, upsert=True)
    
    # mesures à plat (cf. fichier d'origine)
    for m in l['mesures']:
        mesures.update_one(
            {"id": l['_id'], 'date': m['date'] },
            { '$set': {
                "id": l['_id'], "service":l['service'], "sigle":l['sigle'], "nom":l['nom'],
                'date': m['date'], 'ponctualite': m['ponctualite'], 'satisfaction': m['satisfaction']
            }},
            upsert=True
        )

<div id="contenu_des_collections"></div>

In [5]:
print(transiliens.count_documents({}), mesures.count_documents({}))

13 416


In [13]:
import pprint
pprint.pprint(mesures.find_one())

{'_id': ObjectId('6377b94049977c26af2efc93'),
 'date': '2013-01',
 'id': 'TRA_1',
 'nom': 'RER A',
 'ponctualite': 83.6,
 'satisfaction': 5.1,
 'service': 'RER',
 'sigle': 'A'}


In [12]:
pprint.pprint(transiliens.find_one())

{'_id': 'TRA_1',
 'mesures': [{'date': '2013-01', 'ponctualite': 83.6, 'satisfaction': 5.1},
             {'date': '2013-04', 'ponctualite': 83.3, 'satisfaction': 5.0},
             {'date': '2013-08', 'ponctualite': 88.9, 'satisfaction': 8.0},
             {'date': '2014-02', 'ponctualite': 85.7, 'satisfaction': 6.0},
             {'date': '2015-02',
              'ponctualite': 73.0954862767,
              'satisfaction': 2.7},
             {'date': '2015-03',
              'ponctualite': 71.9550770992,
              'satisfaction': 2.6},
             {'date': '2015-06',
              'ponctualite': 71.1496303121,
              'satisfaction': 2.5},
             {'date': '2013-09', 'ponctualite': 83.3, 'satisfaction': 5.0},
             {'date': '2013-11', 'ponctualite': 80.9, 'satisfaction': 4.2},
             {'date': '2014-05', 'ponctualite': 87.6, 'satisfaction': 7.1},
             {'date': '2014-10', 'ponctualite': 68.8, 'satisfaction': 2.2},
             {'date': '2014-11', 'po

### Recherches sur la structure à plat

Cette requête renvoie l'ensemble des informations d'une ligne de données (cf. SELECT * WHERE ponctualite > 96.2)

In [6]:
found = mesures.find({'ponctualite': {'$gt': 96.2}})
for f in found:
    print(f['nom'], f['date'], f['ponctualite'])

Paris Nord Ouest 2014-05 96.9
Paris Nord Ouest 2015-08 96.3
Paris Nord Ouest 2014-03 96.4
Paris Nord Ouest 2014-09 96.3
RER E 2013-09 96.8
RER E 2015-03 96.3263168075
Paris Montparnasse 2015-08 96.3


<div id="gt">
Requête avec projection : ne renvoie que le nom, la date et la ponctualité
</div>

In [23]:
found = mesures.find({'ponctualite': {'$gt': 96.2}}, {"nom":1, "date":1, "ponctualite":1, "_id":0 })
for f in found:
    print(f)

{'date': '2014-05', 'nom': 'Paris Nord Ouest', 'ponctualite': 96.9}
{'date': '2015-08', 'nom': 'Paris Nord Ouest', 'ponctualite': 96.3}
{'date': '2014-03', 'nom': 'Paris Nord Ouest', 'ponctualite': 96.4}
{'date': '2014-09', 'nom': 'Paris Nord Ouest', 'ponctualite': 96.3}
{'date': '2013-09', 'nom': 'RER E', 'ponctualite': 96.8}
{'date': '2015-03', 'nom': 'RER E', 'ponctualite': 96.3263168075}
{'date': '2015-08', 'nom': 'Paris Montparnasse', 'ponctualite': 96.3}


### Recherches sur la structure arborescente

Cette requête renvoie le nom de la ligne pour les lignes qui présentent une mesure de ponctualité meilleure que 96.2%, ainsi que la date et la valeur de la ponctualité pour l'ensemble des mesures des lignes concernées.

In [8]:
# renvoie toutes les mesures des listes en contenant une de ponctualité > 96.2%
found = transiliens.find(
    {"mesures.ponctualite" : {"$gt": 96.2 }},
    {"nom":1, "mesures.date":1, "mesures.ponctualite": 1 }
)

# traitement en python pour n'afficher que les mesures de ponctualité > 96.2%
for f in found:
    print(f['nom'])
    for m in f['mesures']:
        if m['ponctualite'] > 96.2:
            pprint.pprint(m)

Paris Nord Ouest
{'date': '2014-05', 'ponctualite': 96.9}
{'date': '2015-08', 'ponctualite': 96.3}
{'date': '2014-03', 'ponctualite': 96.4}
{'date': '2014-09', 'ponctualite': 96.3}
RER E
{'date': '2013-09', 'ponctualite': 96.8}
{'date': '2015-03', 'ponctualite': 96.3263168075}
Paris Montparnasse
{'date': '2015-08', 'ponctualite': 96.3}


Cette requête renvoie le nom de la ligne et la première mesure concernée, pour les lignes qui présentent une mesure de ponctualité meilleure que 96.2%. 

In [26]:
# renvoie le nom de la ligne, et la première mesure de ponctualité > 96.2% des ligne qui en contiennent au moins une
found = transiliens.find(
    {"mesures.ponctualite" : {"$gt": 96.2 }},
    {"nom":1, "mesures.$": 1, "_id":0}
    )
for f in found:
    pprint.pprint(f)

{'mesures': [{'date': '2014-05', 'ponctualite': 96.9, 'satisfaction': 31.3}],
 'nom': 'Paris Nord Ouest'}
{'mesures': [{'date': '2013-09', 'ponctualite': 96.8, 'satisfaction': 30.3}],
 'nom': 'RER E'}
{'mesures': [{'date': '2015-08', 'ponctualite': 96.3, 'satisfaction': 26.0}],
 'nom': 'Paris Montparnasse'}


Renvoie le nom de toutes lignes et la première mesure de ponctualité > 96.2% s'il en existe une

In [27]:
found = transiliens.find(
    {},
    {"_id":0, "nom":1, "mesures": {"$elemMatch": { "ponctualite": { "$gt": 96.2 }}}}
)
for f in found:
    pprint.pprint(f)

{'nom': 'RER A'}
{'nom': 'RER B'}
{'mesures': [{'date': '2014-05', 'ponctualite': 96.9, 'satisfaction': 31.3}],
 'nom': 'Paris Nord Ouest'}
{'nom': 'Paris Nord Crépy'}
{'nom': 'Paris Est'}
{'nom': 'RER D'}
{'mesures': [{'date': '2013-09', 'ponctualite': 96.8, 'satisfaction': 30.3}],
 'nom': 'RER E'}
{'nom': 'Paris Saint-Lazare Nord'}
{'mesures': [{'date': '2015-08', 'ponctualite': 96.3, 'satisfaction': 26.0}],
 'nom': 'Paris Montparnasse'}
{'nom': 'Paris Saint-Lazare Sud'}
{'nom': 'La Verrière - La Défense'}
{'nom': 'Paris Sud Est'}
{'nom': 'RER C'}


<div id="aggregation">
Requête avec aggrégation pour renvoyer le nom des lignes avec des mesures de ponctualité > 96% et les mesures correspondantes, triées par date
</div>

In [48]:
found = transiliens.aggregate([
    {
        "$match": {"mesures.ponctualite" : {"$gt": 96.2 }}
    },
    {
         "$project": {
             "mesures": {
                 "$filter": {
                     "input": "$mesures",
                     "cond": { "$gt": ["$$this.ponctualite", 96.2] }
                 },
             },
             "nom": 1,
             "_id": 0
        }
    },
    {
         "$project": {
             "mesures": {
                 "$sortArray": {
                     "input": "$mesures",
                     "sortBy": { "date": 1 }
                 }
             },
            "nom": 1
         }
    }

])
pprint.pprint([f for f in found])

[{'mesures': [{'date': '2014-03', 'ponctualite': 96.4, 'satisfaction': 26.8},
              {'date': '2014-05', 'ponctualite': 96.9, 'satisfaction': 31.3},
              {'date': '2014-09', 'ponctualite': 96.3, 'satisfaction': 26.0},
              {'date': '2015-08', 'ponctualite': 96.3, 'satisfaction': 26.0}],
  'nom': 'Paris Nord Ouest'},
 {'mesures': [{'date': '2013-09', 'ponctualite': 96.8, 'satisfaction': 30.3},
              {'date': '2015-03',
               'ponctualite': 96.3263168075,
               'satisfaction': 26.2}],
  'nom': 'RER E'},
 {'mesures': [{'date': '2015-08', 'ponctualite': 96.3, 'satisfaction': 26.0}],
  'nom': 'Paris Montparnasse'}]


In [46]:
pprint.pprint(found)

<div id="tri"></div>
### Tri

In [24]:
found = mesures.find({'ponctualite': {'$gt': 96.2}}, {"nom":1, "date":1, "ponctualite":1, "_id":0 }).sort("date",1)
for f in found:
    print(f)

{'date': '2013-09', 'nom': 'RER E', 'ponctualite': 96.8}
{'date': '2014-03', 'nom': 'Paris Nord Ouest', 'ponctualite': 96.4}
{'date': '2014-05', 'nom': 'Paris Nord Ouest', 'ponctualite': 96.9}
{'date': '2014-09', 'nom': 'Paris Nord Ouest', 'ponctualite': 96.3}
{'date': '2015-03', 'nom': 'RER E', 'ponctualite': 96.3263168075}
{'date': '2015-08', 'nom': 'Paris Nord Ouest', 'ponctualite': 96.3}
{'date': '2015-08', 'nom': 'Paris Montparnasse', 'ponctualite': 96.3}


In [25]:
found = mesures.find(
    {'ponctualite': {'$gt': 96.2}},
    {"nom":1, "date":1, "ponctualite":1, "_id":0 }
).sort([("nom",1),("date",1)])
for f in found:
    print(f)

{'date': '2015-08', 'nom': 'Paris Montparnasse', 'ponctualite': 96.3}
{'date': '2014-03', 'nom': 'Paris Nord Ouest', 'ponctualite': 96.4}
{'date': '2014-05', 'nom': 'Paris Nord Ouest', 'ponctualite': 96.9}
{'date': '2014-09', 'nom': 'Paris Nord Ouest', 'ponctualite': 96.3}
{'date': '2015-08', 'nom': 'Paris Nord Ouest', 'ponctualite': 96.3}
{'date': '2013-09', 'nom': 'RER E', 'ponctualite': 96.8}
{'date': '2015-03', 'nom': 'RER E', 'ponctualite': 96.3263168075}
